In [11]:
import pandas as pd
import yfinance as yf
import talib as talib
import matplotlib.pyplot as plt
import numpy as np
from backtesting import Strategy, Backtest

In [3]:
"""
    Market: Given by user
    Entry: Price above 500-day SMA & 10-period RSI below 30 
    Exit: close price below 500-day SMA
"""

'\n    Market: S&P 500\n    Entry: Price above 500-day SMA & 10-period RSI below 30 \n    Exit: close price below 500-day SMA\n'

In [4]:
pd.options.mode.chained_assignment = None

In [60]:
# create the DataFrame using YahooFinance
df = yf.download('BTC-USD', start='1995-01-01')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2022-03-15,39664.250000,39794.628906,38310.210938,39338.785156,39338.785156,23934000868
2022-03-16,39335.570312,41465.453125,39022.347656,41143.929688,41143.929688,39616916192
2022-03-17,41140.843750,41287.535156,40662.871094,40951.378906,40951.378906,22009601093


In [61]:
class MA_RSI(Strategy):
    sma_window = 500
    rsi_window = 10
    atr_window = 14
    stop_loss = 5
    
    
    def init(self):
        self.sma = self.I(talib.SMA, self.data.Close, self.sma_window)
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_window)
        
    def next(self):
        if self.data.Close > self.sma and self.rsi < 25:
            self.buy(sl = self.data.Low - self.data.Low * self.stop_loss/100)
        elif self.data.Close < self.sma:
            self.position.close()
            
        
        
        
                

In [62]:
bt = Backtest(df, MA_RSI, cash=10000, commission=0.04, exclusive_orders=True)
stats = bt.run()
print(stats)
bt.plot(plot_volume=False)


/tmp/ipykernel_456/1916854544.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, MA_RSI, cash=10000, commission=0.04, exclusive_orders=True)


Start                     2014-09-17 00:00:00
End                       2022-03-19 00:00:00
Duration                   2740 days 00:00:00
Exposure Time [%]                   20.138636
Equity Final [$]                 13480.607725
Equity Peak [$]                  74502.381689
Return [%]                          34.806077
Buy & Hold Return [%]             9079.801655
Return (Ann.) [%]                     4.05729
Volatility (Ann.) [%]               32.691696
Sharpe Ratio                         0.124108
Sortino Ratio                        0.183432
Calmar Ratio                          0.04953
Max. Drawdown [%]                  -81.915444
Avg. Drawdown [%]                   -7.797449
Max. Drawdown Duration     1660 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                                   25
Win Rate [%]                             12.0
Best Trade [%]                     249.101321
Worst Trade [%]                    -12.476088
Avg. Trade [%]                    

Row(id='12421', ...)

/usr/bin/sensible-browser: 8: /mnt/c/Program: not found
[6637:6674:0319/114359.099525:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[6637:6674:0319/114359.101103:ERROR:bus.cc(397)] Failed to connect to the bus: Failed to connect to socket /var/run/dbus/system_bus_socket: No such file or directory
[6637:6673:0319/114359.159550:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[6637:6673:0319/114359.159618:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[6637:6673:0319/114359.290222:ERROR:bus.cc(397)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[6637:6673:0319/114359.290633:ERROR:bus.cc(397)] Fa

In [35]:
op = bt.optimize(
    sma_window = range(100,500),
    rsi_window = range(10,14),
    atr_window = range(10,20),
    stop_loss = range(5,15),
    maximize = 'Equity Final [$]'
)
print(op)
print('*'*100)
print(f'Stats Strategy: {stats._strategy}')
print('*'*100)

/home/samueltg92/miniconda3/envs/TRADING/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 160000 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/534 [00:00<?, ?it/s]

Start                     2014-09-17 00:00:00
End                       2022-03-19 00:00:00
Duration                   2740 days 00:00:00
Exposure Time [%]                   27.325794
Equity Final [$]                219620.457495
Equity Peak [$]                 468143.340308
Return [%]                        2096.204575
Buy & Hold Return [%]             9030.020104
Return (Ann.) [%]                   50.890285
Volatility (Ann.) [%]               63.973015
Sharpe Ratio                         0.795496
Sortino Ratio                        1.969602
Calmar Ratio                         0.958622
Max. Drawdown [%]                   -53.08692
Avg. Drawdown [%]                   -8.781864
Max. Drawdown Duration     1554 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                     354.856833
Worst Trade [%]                     44.506401
Avg. Trade [%]                    